In [1]:
# My personal version of Cholla plot_sod.py plotting script
import h5py
%pylab inline
pylab.rcParams.update({'font.size': 18})
pylab.rcParams.update({'font.family': 'serif'})
pylab.rcParams.update({'mathtext.default':'regular'})
pylab.rcParams.update({'mathtext.fontset':'stixsans'})
pylab.rcParams.update({'axes.linewidth': 1.5})
pylab.rcParams.update({'xtick.direction':'in'})
pylab.rcParams.update({'xtick.major.size': 5})
pylab.rcParams.update({'xtick.major.width': 1.25 })
pylab.rcParams.update({'xtick.minor.size': 2.5})
pylab.rcParams.update({'xtick.minor.width': 1.25 })
pylab.rcParams.update({'ytick.direction':'in'})
pylab.rcParams.update({'ytick.major.size': 5})
pylab.rcParams.update({'ytick.major.width': 1.25 })
pylab.rcParams.update({'ytick.minor.size': 2.5})
pylab.rcParams.update({'ytick.minor.width': 1.25 })

Populating the interactive namespace from numpy and matplotlib


In [19]:
dnamein = "./sod_1D/"
dnameout = "./sod_1D/"
i = 1
f = h5py.File(dnamein + str(i) + ".h5", "r")
head = f.attrs # Allows dictionary-style access to an HDF5 object's attributes.
print(f"Keywords: {head.keys()}")

Keywords: <KeysViewHDF5 ['bounds', 'density_unit', 'dims', 'domain', 'dt', 'dx', 'energy_unit', 'gamma', 'length_unit', 'mass_unit', 'n_fields', 'n_step', 't', 'time_unit', 'velocity_unit']>
